In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from tensorflow.keras import layers
from tensorflow import keras

In [2]:
ROOT = 'E:/skia_projects/3d_facial_landmark/implementation_1/data_new/temp'

In [3]:
train_data = pd.read_csv(os.path.join(ROOT, 'train_balanced.csv'), delimiter=',', 
                         index_col=False, names=['X', 'Y', 'Z', 'label'])

In [4]:
print(train_data)

               X         Y         Z  label
0       0.036309 -0.000011  0.000140      0
1       0.051504  0.000000  0.000126      0
2       0.067061  0.000000  0.000126      0
3       0.091582  0.000000  0.000126      0
4       0.108457  0.000000  0.000126      0
...          ...       ...       ...    ...
120853  0.074577 -0.079726  0.145563      1
120854 -0.025355  0.212418  0.072108      1
120855  0.159572 -0.067101  0.045794      1
120856 -0.187689  0.160424  0.067511      1
120857 -0.016911  0.202529  0.087553      1

[120858 rows x 4 columns]


In [5]:
labels = train_data.pop('label').values
print(labels)

[0 0 0 ... 1 1 1]


In [6]:
labels = tf.one_hot(labels, 2)
print(labels)

tf.Tensor(
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]], shape=(120858, 2), dtype=float32)


In [7]:
features = train_data.values
print(features)

[[ 3.6309000e-02 -1.1000000e-05  1.4000000e-04]
 [ 5.1504000e-02  0.0000000e+00  1.2600000e-04]
 [ 6.7061000e-02  0.0000000e+00  1.2600000e-04]
 ...
 [ 1.5957201e-01 -6.7101000e-02  4.5794000e-02]
 [-1.8768901e-01  1.6042399e-01  6.7511000e-02]
 [-1.6911000e-02  2.0252900e-01  8.7553000e-02]]


In [8]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels))

for data, labels in dataset.take(1):
    print(data)
    print(labels)

tf.Tensor([ 3.6309e-02 -1.1000e-05  1.4000e-04], shape=(3,), dtype=float64)
tf.Tensor([1. 0.], shape=(2,), dtype=float32)


In [9]:
train_ds = dataset.shuffle(len(dataset)).batch(32)

In [11]:
model = tf.keras.Sequential([
    layers.InputLayer(input_shape=(3,)),
    layers.Dense(10, activation='relu'),
    layers.Dense(20, activation='relu'),
    layers.Dense(10, activation='relu'),
    layers.Dense(2, activation='softmax')
])

model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer = 'adam', metrics=['accuracy'])

In [12]:
model.fit(train_ds, epochs=50)

Epoch 1/50
3777/3777 [==============================] - 13s 3ms/step - loss: 0.2106 - accuracy: 0.6543
Epoch 2/50
3777/3777 [==============================] - 11s 3ms/step - loss: 0.1818 - accuracy: 0.7253
Epoch 3/50
3777/3777 [==============================] - 11s 3ms/step - loss: 0.1691 - accuracy: 0.7525
Epoch 4/50
3777/3777 [==============================] - 11s 3ms/step - loss: 0.1595 - accuracy: 0.7724
Epoch 5/50
3777/3777 [==============================] - 11s 3ms/step - loss: 0.1562 - accuracy: 0.7776
Epoch 6/50
3777/3777 [==============================] - 11s 3ms/step - loss: 0.1532 - accuracy: 0.7809
Epoch 7/50
3777/3777 [==============================] - 11s 3ms/step - loss: 0.1520 - accuracy: 0.7845
Epoch 8/50
3777/3777 [==============================] - 11s 3ms/step - loss: 0.1503 - accuracy: 0.7861
Epoch 9/50
3777/3777 [==============================] - 11s 3ms/step - loss: 0.1493 - accuracy: 0.7874
Epoch 10/50
3777/3777 [==============================] - 11s 3ms/step - l

In [13]:
model.save("model.h5")

In [14]:
test_data = pd.read_csv(os.path.join(ROOT, 'test.csv'), delimiter=',', index_col=False, usecols=[0,1,2])

In [15]:
print(test_data)

      -0.14728101  0.00526800  0.01012900
0       -0.079014    0.005291    0.000879
1       -0.095757    0.005285    0.000967
2       -0.115664    0.005279    0.001055
3       -0.055800    0.005268    0.001758
4       -0.148287    0.005269    0.019786
...           ...         ...         ...
9594    -0.034393    0.182729    0.123804
9595    -0.062175    0.182714    0.144613
9596    -0.048653    0.182730    0.137966
9597    -0.056554    0.182730    0.142383
9598    -0.075801    0.182728    0.146889

[9599 rows x 3 columns]


In [16]:
#test_data = np.array(test_data)

In [17]:
class_names = ['background', 'landmark']

In [18]:
prediction = model.predict(test_data)

In [19]:
print(prediction)

[[1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 ...
 [0.82486075 0.1751392 ]
 [0.76075554 0.23924445]
 [0.7799684  0.2200317 ]]


In [20]:
predicted_class_indice = np.argmax(prediction, axis=-1)

In [21]:
f = lambda x: class_names[int(x)]
vf = np.vectorize(f)

In [22]:
predicted_class = vf(predicted_class_indice)

In [23]:
print(predicted_class.dtype)

<U10


In [24]:
print(predicted_class.shape, test_data.shape)

(9599,) (9599, 3)


In [25]:
data = np.column_stack((test_data, predicted_class))

In [26]:
print(data.shape)

(9599, 4)


In [27]:
df = pd.DataFrame({'X': data[:,0], 'Y': data[:,1], 'Z':data[:,2], 'Predictions':data[:,3]})
results = df.to_csv(os.path.join(ROOT, 'results.csv'), index=False)

In [28]:
df = pd.DataFrame({'X': data[:,0], 'Y': data[:,1], 'Z':data[:,2], 'Predictions':data[:,3]})
results = df[df.Predictions != 'background']
results = results.to_csv(os.path.join(ROOT, 'results_landmarks.csv'), index=False)

In [29]:
# Write only points
df = pd.DataFrame({'X': data[:,0], 'Y': data[:,1], 'Z':data[:,2], 'Predictions':data[:,3]})
results = df[df.Predictions != 'background']
results = results.to_csv(os.path.join(ROOT, 'results_landmarks_points.csv'), index=False, 
                         columns =('X', 'Y', 'Z'), header=False)